# Tensorflow JS - Binary classification
## Predict if AFI goes for a run today

In [1]:
// First import the Luxon library
import { DateTime } from 'npm:luxon'

In [2]:
const startDate = DateTime.fromObject({day: 1, month: 1, year: 2024})
const endDate = DateTime.fromObject({day: 31, month: 10, year: 2024})

// AFI goes for a run every Monday, Wednesday and Friday
// Except for last day of the month

let loopDate = startDate;
const learningData = [];

while (loopDate <= endDate) {
  learningData.push({
    dayStr: loopDate.toFormat('dd-MM-yyyy'),
    weekday: loopDate.weekday,
    isLastDayOfMonth: loopDate.day === loopDate.daysInMonth,
    goingForRun: (loopDate.weekday === 1 || loopDate.weekday === 3 || loopDate.weekday === 5) && loopDate.day !== loopDate.daysInMonth,
  });    
  loopDate = loopDate.plus({days: 1});
}


DateTime { ts: 2024-11-01T00:00:00.000+01:00, zone: Europe/Zurich, locale: en-US }

## Preparation of data for model fitting


In [3]:
// If you have questions https://www.afiether.com/contact
// If you want to be up to date https://www.afiether.com/subscribe

In [4]:
import pl from 'npm:nodejs-polars'

In [5]:
const learningDf = pl.readRecords(learningData)
const modelDf = learningDf
  .select(
    pl.col('weekday'),
    pl.when(pl.col('isLastDayOfMonth').eq(true))
      .then(pl.lit(1))
      .otherwise(pl.lit(0))
      .alias('isLastDayOfMonth'),
    pl.when(pl.col('goingForRun').eq(true))
      .then(pl.lit(1))
      .otherwise(pl.lit(0))
      .alias('goingForRun'),
            
  );
modelDf

weekday,isLastDayOfMonth,goingForRun
1,0,1
2,0,0
3,0,1
4,0,0
5,0,1
6,0,0
7,0,0
1,0,1
2,0,0
3,0,1


In [6]:
const inputVector = modelDf.toRecords().map(_ => 
  [_.weekday, _.isLastDayOfMonth]
);
const outputVector = modelDf.toRecords().map(_ => 
  [_.goingForRun]
);

In [7]:
outputVector

[
  [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 0 ], [ 1 ],
  [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 0 ], [ 1 ], [ 0 ],
  [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ],
  [ 0 ], [ 1 ], [ 0 ], [ 0 ], [ 1 ], [ 0 ], [ 0 ], [ 0 ],
  [ 1 ], [ 0 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ],
  [ 0 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ],
  [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 0 ],
  [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 0 ], [ 1 ],
  [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 0 ], [ 1 ], [ 0 ],
  [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ],
  [ 0 ], [ 1 ], [ 0 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ],
  [ 1 ], [ 0 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ], [ 0 ], [ 1 ],
  [ 0 ], [ 0 ], [ 1 ], [ 0 ],
  ... 205 more items
]

## Instatiate and train the model

In [8]:
// Use 'npm:@tensorflow/tfjs-node' if you don't have CUDA
import tf from 'npm:@tensorflow/tfjs-node-gpu'

const model = tf.sequential();

// Start with the input layer
model.add(tf.layers.dense({ units: 16, inputShape: [2], activation: 'relu'}));

// Secondary learning hidden layer
model.add(tf.layers.dense({ units: 32, activation: 'relu'}));

// The output layer
model.add(tf.layers.dense({ units: 1, activation: 'sigmoid'}));

// Compile the model in memory
model.compile({loss: 'binaryCrossentropy', optimizer: 'adam'})

// Define tensors
const xs = tf.tensor2d(inputVector, [inputVector.length, 2]);
const ys = tf.tensor2d(outputVector, [outputVector.length, 1]);

const _np = await model.fit(xs, ys, {epochs: 3000, verbose: 0, validationSplit: 0.3});

## Test the model by predictions

In [18]:
const predictInputs = [
  {p: [1, 0], d: 'Monday, not the last day of month'},
  {p: [2, 0], d:'Tuesday, not the last day of month'},
  {p: [3, 1], d: 'Wednesday, IS the last day of month'},
  {p: [4, 1], d:'Thursday, IS the last day of month'},
  {p: [5, 0], d: 'Friday, not the last day of month  '},
  {p: [5, 1], d: 'Friday, IS the last day of month  '},
  {p: [3, 0], d: 'Wednesday, not the last day of month'},
  {p: [4, 0], d:'Thursday, not the last day of month'},
  {p: [6, 0], d:'Saturday, not the last day of month'},
  {p: [7, 0], d:'Sunday, not the last day of month'},
]

const predictTensor = tf.tensor2d(predictInputs.map(_ => _.p), [predictInputs.length, 2])
const prediction = await model.predict(predictTensor).data();

const threshold = 0.6;

predictInputs.map((pred, i) => ({
  pred: prediction[i] > threshold ? 'RUNNING' : 'NOT RUNNING',
  desc: pred.d  
}))


[
  { pred: "RUNNING", desc: "Monday, not the last day of month" },
  { pred: "NOT RUNNING", desc: "Tuesday, not the last day of month" },
  { pred: "NOT RUNNING", desc: "Wednesday, IS the last day of month" },
  { pred: "NOT RUNNING", desc: "Thursday, IS the last day of month" },
  { pred: "RUNNING", desc: "Friday, not the last day of month  " },
  { pred: "NOT RUNNING", desc: "Friday, IS the last day of month  " },
  { pred: "RUNNING", desc: "Wednesday, not the last day of month" },
  { pred: "RUNNING", desc: "Thursday, not the last day of month" },
  { pred: "NOT RUNNING", desc: "Saturday, not the last day of month" },
  { pred: "NOT RUNNING", desc: "Sunday, not the last day of month" }
]